# Running the Luminosity-Temperature-Radius pipeline

We will use the XGA LTR pipeline to try to determine overdensity radii, and the global temperature and luminosity measurements within them. This process is [fully described here](https://xga.readthedocs.io/en/latest/notebooks/pipeline_tutorials/LT_pipeline.html). This should be relatively successful, even given the low surface brightness of the groups, as it easier to get temperature constraints on lower-temperature systems than hotter systems when using XMM.

Visualisations will also be generated here, to demonstrate the regions that were determined to be contaminating sources and masked out, as some adjustment may be required given the nature of these groups.

## Import statements

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from astropy.units import Quantity
import os
from shutil import rmtree

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 40
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/lum_temp_rad_pipeline')[0]+'xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.tools import luminosity_temperature_pipeline
from xga.relations.clusters.RT import arnaud_r2500

%matplotlib inline

## Reading the sample file

Reading in the positions, names, and redshifts of the groups - the pipeline expects position columns to be called 'ra' and 'dec', so those are renamed:

In [2]:
samp = pd.read_csv('../../sample_files/init_group_info.csv')
samp = samp.rename(columns={'est_ra': 'ra', 'est_dec': 'dec'})
samp

,name,ra,dec,redshift
0,25124,243.629055,26.730240,0.186
1,44739,229.793620,28.331750,0.118
2,19670A,150.256710,50.793942,0.134
3,19670B,150.214920,50.805014,0.134
4,12833,129.496800,44.248700,0.145
5,44858,230.112000,28.887750,0.127
6,PHL1811,328.769920,-9.588805,0.077


## Running the pipeline (twice)

The LTR will be run twice, targeting two different overdensity radii. The pipeline runs will save the results to disk (as we are passing paths for the results and radii history files), as well as returning the results and the XGA ClusterSample object generated by the procedure.

### $R_{500}$

The first overdensity radius we are targeting is $R_{500}$, which is the default for this pipeline. The Arnaud et al. (2005) $R_{500}$-$T_{\rm{X}}$ is used for the process, and the starting radius is set to 200 kpc. Metallicity, nH, and redshift are all frozen during the spectral fitting process.

In [3]:
res_file = '../../outputs/results/ltr_r500_pipeline_results.csv'
rhist_file = '../../outputs/results/radii_r500_history.csv'

srcs, results, rad_hist = luminosity_temperature_pipeline(samp, Quantity(200, 'kpc'), timeout=Quantity(4, 'hr'), 
                                                          save_samp_results_path=res_file,
                                                          save_rad_history_path=rhist_file, 
                                                          core_excised=True)

Generating products of type(s) expmap: 100%|██████████| 2/2 [00:00<00:00,  8.67it/s]
/mnt/ufs18/home-218/turne540/code/XGA/xga/samples/extended.py:237: UserWarning: Non-fatal warnings occurred during the declaration of some sources, to access them please use the suppressed_warnings property of this sample.
  self._check_source_warnings()
Running XSPEC Fits: 100%|██████████| 7/7 [00:26<00:00,  3.75s/it]


### $R_{2500}$

The other overdensity radius we are targeting is $R_{2500}$. The Arnaud et al. (2005) $R_{2500}$-$T_{\rm{X}}$ is used for the process. As the ClusterSample in this run is completely separate from the $R_{500}$ run, it is possible that slightly different regions have been selected as contaminating sources, this would be due to XGA ClusterSamples allowing point sources in the core region to not be masked out, in case they are misidentified cool-cores.

In [4]:
res_file = '../../outputs/results/ltr_r2500_pipeline_results.csv'
rhist_file = '../../outputs/results/radii_r2500_history.csv'

srcs_r2500, results_r2500, \
    rad_hist_r2500 = luminosity_temperature_pipeline(samp, Quantity(200, 'kpc'), timeout=Quantity(4, 'hr'), 
                                                     rad_temp_rel=arnaud_r2500, save_samp_results_path=res_file,
                                                     save_rad_history_path=rhist_file)

Setting up Galaxy Clusters: 100%|██████████| 7/7 [00:04<00:00,  1.45it/s]
/mnt/ufs18/home-218/turne540/code/XGA/xga/samples/extended.py:237: UserWarning: Non-fatal warnings occurred during the declaration of some sources, to access them please use the suppressed_warnings property of this sample.
  self._check_source_warnings()
/mnt/ufs18/home-218/turne540/code/XGA/xga/xspec/run.py:186: UserWarning: All XSPEC operations had already been run.
  warnings.warn("All XSPEC operations had already been run.")
Running XSPEC Fits: 100%|██████████| 7/7 [00:31<00:00,  4.49s/it]
/mnt/ufs18/home-218/turne540/code/XGA/xga/products/relation.py:704: UserWarning: Some of the x values you have passed are outside the validity range of this relation (1.0-12.0keV).
  warn("Some of the x values you have passed are outside the validity range of this relation "
Running XSPEC Fits: 100%|██████████| 2/2 [00:14<00:00,  7.26s/it]
/mnt/ufs18/home-218/turne540/code/XGA/xga/tools/clusters/LT.py:291: UserWarning: The 

### $R_{500}$ - with metallicity free

The first overdensity radius we are targeting is $R_{500}$, which is the default for this pipeline. The Arnaud et al. (2005) $R_{500}$-$T_{\rm{X}}$ is used for the process, and the starting radius is set to 200 kpc. nH, and redshift are all frozen during the spectral fitting process.

In [3]:
res_file = '../../outputs/results/ltr_r500_metfree_pipeline_results.csv'
rhist_file = '../../outputs/results/radii_metfree_r500_history.csv'

srcs_metfree, results_metfree, rad_hist_metfree = luminosity_temperature_pipeline(samp, Quantity(200, 'kpc'), 
                                                                                  timeout=Quantity(4, 'hr'), 
                                                                                  save_samp_results_path=res_file, 
                                                                                  save_rad_history_path=rhist_file, 
                                                                                  core_excised=True, freeze_met=False)

Setting up Galaxy Clusters: 100%|██████████| 7/7 [00:04<00:00,  1.73it/s]
/mnt/ufs18/home-218/turne540/code/XGA/xga/samples/extended.py:237: UserWarning: Non-fatal warnings occurred during the declaration of some sources, to access them please use the suppressed_warnings property of this sample.
  self._check_source_warnings()
Running XSPEC Fits: 100%|██████████| 1/1 [00:30<00:00, 30.69s/it]
/mnt/ufs18/home-218/turne540/code/XGA/xga/tools/clusters/LT.py:297: UserWarning: The radius measurement process reached the maximum number of iterations; as such one or more clusters may have unconverged radii.
  warn("The radius measurement process reached the maximum number of iterations; as such one or more clusters "
Running XSPEC Fits: 100%|██████████| 7/7 [00:51<00:00,  7.31s/it]


### $R_{2500}$ - with metallicity free

The other overdensity radius we are targeting is $R_{2500}$. The Arnaud et al. (2005) $R_{2500}$-$T_{\rm{X}}$ is used for the process. As the ClusterSample in this run is completely separate from the $R_{500}$ run, it is possible that slightly different regions have been selected as contaminating sources, this would be due to XGA ClusterSamples allowing point sources in the core region to not be masked out, in case they are misidentified cool-cores.

In [4]:
res_file = '../../outputs/results/ltr_r2500_metfree_pipeline_results.csv'
rhist_file = '../../outputs/results/radii_r2500_metfree_history.csv'

srcs_r2500_metfree, results_r2500_metfree, \
    rad_hist_r2500_metfree = luminosity_temperature_pipeline(samp, Quantity(200, 'kpc'), timeout=Quantity(4, 'hr'), 
                                                             rad_temp_rel=arnaud_r2500, 
                                                             save_samp_results_path=res_file,
                                                             save_rad_history_path=rhist_file,
                                                             freeze_met=False)

Setting up Galaxy Clusters: 100%|██████████| 7/7 [00:04<00:00,  1.70it/s]
/mnt/ufs18/home-218/turne540/code/XGA/xga/samples/extended.py:237: UserWarning: Non-fatal warnings occurred during the declaration of some sources, to access them please use the suppressed_warnings property of this sample.
  self._check_source_warnings()
Running XSPEC Fits: 100%|██████████| 7/7 [01:06<00:00,  9.46s/it]
/mnt/ufs18/home-218/turne540/code/XGA/xga/products/relation.py:704: UserWarning: Some of the x values you have passed are outside the validity range of this relation (1.0-12.0keV).
  warn("Some of the x values you have passed are outside the validity range of this relation "
Running XSPEC Fits: 100%|██████████| 7/7 [00:55<00:00,  7.91s/it]
/mnt/ufs18/home-218/turne540/code/XGA/xga/samples/extended.py:599: UserWarning: Those spectra have no associated XSPEC fit to 19670A
  warn(str(err))
Running XSPEC Fits: 100%|██████████| 1/1 [00:32<00:00, 32.29s/it]
/mnt/ufs18/home-218/turne540/code/XGA/xga/tools

## Pipeline results

We take a look at the tables of results output by the pipeline runs:

### $R_{500}$

In [6]:
results

,name,ra,dec,redshift,r500,Tx500,Tx500-,Tx500+,Lx500_0.5-2.0,Lx500_0.5-2.0-,Lx500_0.5-2.0+,Lx500_0.01-100.0,Lx500_0.01-100.0-,Lx500_0.01-100.0+,Tx500ce,Tx500ce-,Tx500ce+,Lx500ce_0.5-2.0,Lx500ce_0.5-2.0-,Lx500ce_0.5-2.0+,Lx500ce_0.01-100.0,Lx500ce_0.01-100.0-,Lx500ce_0.01-100.0+
0,25124,243.629055,26.730240,0.186,884.907276,4.00041,0.226145,0.249232,7.126952e+43,1.664257e+42,1.180090e+42,2.093496e+44,7.035722e+42,7.733165e+42,4.18235,0.347023,0.446068,4.401046e+43,1.054248e+42,1.092011e+42,1.320009e+44,9.289548e+42,4.509234e+42
1,44739,229.793620,28.331750,0.118,441.929890,1.04939,0.076687,0.077842,2.583824e+42,3.222766e+41,3.508307e+41,4.570903e+42,3.987661e+41,5.525575e+41,1.00346,0.075609,0.093716,2.170778e+42,2.687851e+41,2.245088e+41,3.763344e+42,5.451670e+41,3.527519e+41
2,19670A,150.256710,50.793942,0.134,567.577741,1.81338,0.322468,0.704376,2.017193e+42,3.185290e+41,4.563993e+41,4.386489e+42,8.506115e+41,1.080462e+42,1.93961,0.402115,0.940446,1.571398e+42,3.035652e+41,5.479817e+41,3.489346e+42,8.563566e+41,1.155577e+42
3,19670B,150.214920,50.805014,0.134,574.829078,2.14102,0.278732,0.478511,5.330541e+42,4.116645e+41,3.671380e+41,1.221776e+43,1.329723e+42,1.038647e+42,2.13246,0.297071,0.511541,4.731718e+42,3.637689e+41,4.076409e+41,1.083089e+43,1.155112e+42,1.365707e+42
4,12833,129.496800,44.248700,0.145,499.965789,1.42217,0.090660,0.125263,3.945332e+42,3.237082e+41,2.217431e+41,7.933681e+42,7.779298e+41,9.311470e+41,1.57358,0.188266,0.322650,2.857480e+42,3.075853e+41,4.402722e+41,5.944700e+42,6.989975e+41,8.226678e+41
5,44858,230.112000,28.887750,0.127,767.772441,2.86648,0.248362,0.306101,2.707709e+43,9.089906e+41,9.728542e+41,6.901237e+43,4.605478e+42,4.373647e+42,2.78504,0.324367,0.387647,1.962684e+43,1.133694e+42,1.009324e+42,4.946080e+43,3.795027e+42,5.370238e+42
6,PHL1811,328.769920,-9.588805,0.077,563.128423,1.69519,0.057591,0.077226,7.932953e+42,4.678598e+41,3.547589e+41,1.791979e+43,1.049040e+42,9.380862e+41,1.70709,0.083884,0.125078,5.165886e+42,3.798389e+41,2.919015e+41,1.169284e+43,7.998273e+41,5.463086e+41


In [9]:
rad_hist

,0,1,2,3,4,5,6,converged
25124,200.0,870.693815,883.430404,887.749298,884.907276,884.907276,884.907276,True
44739,200.0,477.857955,505.399068,591.829633,430.529500,441.929890,441.929890,True
19670A,200.0,421.405415,504.177256,571.579737,567.577741,567.577741,567.577741,True
19670B,200.0,834.334858,557.142229,665.530113,501.016564,610.679034,574.829078,True
12833,200.0,495.914999,498.862200,499.253222,499.965789,499.965789,499.965789,True
44858,200.0,799.519256,779.826961,771.953074,767.772441,767.772441,767.772441,True
PHL1811,200.0,603.163704,556.387866,581.477006,563.128423,563.128423,563.128423,True


### $R_{500}$ - with metallicity free

In [5]:
results_metfree

,name,ra,dec,redshift,r500,Tx500,Tx500-,Tx500+,Lx500_0.5-2.0,Lx500_0.5-2.0-,Lx500_0.5-2.0+,Lx500_0.01-100.0,Lx500_0.01-100.0-,Lx500_0.01-100.0+,Zmet500,Zmet500-,Zmet500+,Tx500ce,Tx500ce-,Tx500ce+,Lx500ce_0.5-2.0,Lx500ce_0.5-2.0-,Lx500ce_0.5-2.0+,Lx500ce_0.01-100.0,Lx500ce_0.01-100.0-,Lx500ce_0.01-100.0+,Zmet500ce,Zmet500ce-,Zmet500ce+
0,25124,243.629055,26.730240,0.186,877.334608,3.89367,0.232046,0.251826,7.164015e+43,1.557383e+42,1.265250e+42,2.068589e+44,7.800382e+42,5.925892e+42,0.178139,0.071397,0.076156,3.812650,0.321356,0.370420,4.480495e+43,1.003176e+42,1.980200e+42,1.272865e+44,5.184877e+42,9.464085e+42,1.996150e-08,1.996150e-08,0.048032
1,44739,229.793620,28.331750,0.118,427.447211,1.00651,0.085874,0.087752,2.654546e+42,3.772840e+41,2.144730e+41,5.243630e+42,8.462623e+41,4.761213e+41,0.093827,0.039917,0.059464,0.960321,0.106900,0.105126,2.302095e+42,3.382720e+41,1.801591e+41,4.601933e+42,9.686040e+41,4.007213e+41,7.534350e-02,3.785460e-02,0.061384
2,19670A,150.256710,50.793942,0.134,472.693568,1.20623,0.140341,0.145959,1.561950e+42,5.588488e+41,2.232024e+41,3.019472e+42,9.251117e+41,4.926049e+41,0.223101,0.129736,0.280678,1.288110,0.151674,0.199351,1.157190e+42,6.035350e+41,9.460520e+40,2.200230e+42,1.037577e+42,3.523851e+41,3.816340e-01,2.934835e-01,1.006238
3,19670B,150.214920,50.805014,0.134,463.405408,2.51331,0.753050,1.252969,3.413026e+42,4.669260e+41,3.925893e+41,8.301481e+42,1.357410e+42,1.617179e+42,0.172265,0.172265,0.283293,2.149860,0.645260,1.118210,3.054648e+42,4.841062e+41,3.555520e+41,7.053082e+42,1.844118e+42,8.097179e+41,2.096140e-01,1.698182e-01,0.362663
4,12833,129.496800,44.248700,0.145,486.675050,1.38009,0.106989,0.129112,4.038652e+42,5.244466e+41,3.694547e+41,8.250734e+42,1.116856e+42,5.697162e+41,0.196150,0.069983,0.097809,1.424410,0.165052,0.254709,2.963518e+42,5.562651e+41,2.938970e+41,6.195556e+42,1.322989e+42,3.054634e+41,1.482170e-01,7.583443e-02,0.129839
5,44858,230.112000,28.887750,0.127,809.006952,3.27664,0.325102,0.373789,2.886755e+43,1.455047e+42,9.190871e+41,7.802731e+43,4.629418e+42,3.674255e+42,0.545549,0.146224,0.180842,3.082230,0.395042,0.494758,2.097202e+43,1.425246e+42,1.005110e+42,5.512189e+43,6.355002e+42,2.992304e+42,5.075340e-01,1.757521e-01,0.240352
6,PHL1811,328.769920,-9.588805,0.077,565.977429,1.67076,0.084579,0.092643,7.994455e+42,3.744324e+41,2.449503e+41,1.806898e+43,1.068109e+42,7.943141e+41,0.270788,0.050469,0.057573,1.577180,0.107328,0.129503,5.303300e+42,4.191078e+41,2.783739e+41,1.200877e+43,1.160980e+42,8.150568e+41,1.872840e-01,5.023096e-02,0.065677


In [6]:
rad_hist_metfree

,0,1,2,3,4,5,6,7,8,9,10,converged
25124,200.0,864.484035,877.950339,874.176889,877.334608,877.334608,877.334608,877.334608,877.334608,877.334608,877.334608,True
44739,200.0,437.477334,421.667897,419.249446,427.447211,427.447211,427.447211,427.447211,427.447211,427.447211,427.447211,True
19670A,200.0,426.539982,479.709956,503.252784,451.314063,472.693568,472.693568,472.693568,472.693568,472.693568,472.693568,True
19670B,200.0,742.601199,432.524808,592.709630,487.064612,683.051932,467.851009,611.685223,489.323872,673.668483,463.405408,False
12833,200.0,494.315025,488.172352,495.193964,486.675050,486.675050,486.675050,486.675050,486.675050,486.675050,486.675050,True
44858,200.0,800.127970,821.285498,827.220529,809.006952,809.006952,809.006952,809.006952,809.006952,809.006952,809.006952,True
PHL1811,200.0,638.625385,557.350043,574.330733,565.977429,565.977429,565.977429,565.977429,565.977429,565.977429,565.977429,True


### $R_{2500}$

In [7]:
results_r2500

,name,ra,dec,redshift,r2500,Tx2500,Tx2500-,Tx2500+,Lx2500_0.5-2.0,Lx2500_0.5-2.0-,Lx2500_0.5-2.0+,Lx2500_0.01-100.0,Lx2500_0.01-100.0-,Lx2500_0.01-100.0+
0,25124,243.629055,26.730240,0.186,383.611964,3.85456,0.180488,0.194618,5.050797e+43,7.340025e+41,8.609427e+41,1.458347e+44,6.134899e+42,4.749973e+42
1,44739,229.793620,28.331750,0.118,187.610866,1.23133,0.238726,1.867732,4.911886e+41,1.013270e+41,1.263923e+41,9.346805e+41,1.578651e+41,2.747040e+41
2,19670A,150.256710,50.793942,0.134,191.917167,1.34568,0.317819,0.690648,1.714148e+41,8.269795e+40,1.607143e+41,3.377437e+41,2.749913e+41,3.581301e+41
3,19670B,150.214920,50.805014,0.134,276.398870,3.35891,0.666592,0.882834,1.732891e+42,1.954775e+41,1.888720e+41,4.713931e+42,8.418831e+41,6.833755e+41
4,12833,129.496800,44.248700,0.145,223.942125,1.39355,0.086670,0.105231,2.260324e+42,1.925953e+41,1.844811e+41,4.512154e+42,5.456433e+41,6.547234e+41
5,44858,230.112000,28.887750,0.127,321.850020,2.52029,0.176609,0.197428,1.368352e+43,7.075705e+41,4.362858e+41,3.321116e+43,2.242261e+42,2.042962e+42
6,PHL1811,328.769920,-9.588805,0.077,259.119501,1.75546,0.067041,0.076451,5.601866e+42,2.885905e+41,2.814568e+41,1.277724e+43,6.857446e+41,8.816534e+41


In [10]:
rad_hist_r2500

,0,1,2,3,4,5,6,7,8,9,10,converged
25124,200.0,388.231401,384.030316,388.491449,383.611964,383.611964,383.611964,383.611964,383.611964,383.611964,383.611964,True
44739,200.0,215.457849,306.878252,176.106534,220.635529,186.075291,211.378898,309.751957,173.850860,220.196975,187.610866,False
19670A,200.0,190.396886,223.233029,195.036039,191.917167,191.917167,191.917167,191.917167,191.917167,191.917167,191.917167,True
19670B,200.0,372.473947,273.591473,360.484689,276.824353,367.796196,273.836627,366.764790,275.067545,361.755160,276.398870,False
12833,200.0,223.400265,224.171340,224.044620,223.942125,223.942125,223.942125,223.942125,223.942125,223.942125,223.942125,True
44858,200.0,357.220411,329.773771,317.830845,321.850020,321.850020,321.850020,321.850020,321.850020,321.850020,321.850020,True
PHL1811,200.0,270.943519,258.693228,263.702955,259.119501,259.119501,259.119501,259.119501,259.119501,259.119501,259.119501,True


### $R_{2500}$ - with metallicity free

In [7]:
results_r2500_metfree

,name,ra,dec,redshift,r2500,Tx2500,Tx2500-,Tx2500+,Lx2500_0.5-2.0,Lx2500_0.5-2.0-,Lx2500_0.5-2.0+,Lx2500_0.01-100.0,Lx2500_0.01-100.0-,Lx2500_0.01-100.0+,Zmet2500,Zmet2500-,Zmet2500+
0,25124,243.629055,26.730240,0.186,385.685473,3.83568,0.162093,0.177542,5.057808e+43,8.580258e+41,8.256542e+41,1.467342e+44,4.516842e+42,3.680594e+42,0.519449,0.077007,0.082626
1,44739,229.793620,28.331750,0.118,208.910538,1.82020,0.795483,2.367874,6.071436e+41,2.605168e+41,1.658230e+41,1.379969e+42,7.206041e+41,6.305033e+41,0.001771,0.001771,0.466957
2,19670A,150.256710,50.793942,0.134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19670B,150.214920,50.805014,0.134,305.336349,2.34755,0.573709,0.932660,2.171640e+42,2.549630e+41,2.496837e+41,5.143428e+42,8.438422e+41,5.005873e+41,0.262433,0.169005,0.411290
4,12833,129.496800,44.248700,0.145,225.235004,1.42142,0.099140,0.134105,2.258968e+42,3.092120e+41,1.966346e+41,4.449311e+42,5.947840e+41,3.080084e+41,0.411702,0.138728,0.226330
5,44858,230.112000,28.887750,0.127,326.634769,2.67258,0.203015,0.221600,1.376012e+43,5.603123e+41,7.582435e+41,3.409997e+43,2.045108e+42,2.103854e+42,0.358755,0.087839,0.103763
6,PHL1811,328.769920,-9.588805,0.077,269.027191,1.81382,0.090345,0.101098,5.551840e+42,3.803461e+41,2.699054e+41,1.259063e+43,8.170027e+41,6.118237e+41,0.432920,0.081925,0.100552


In [8]:
rad_hist_r2500_metfree

,0,1,2,3,4,5,6,7,8,9,10,converged
25124,200.0,385.510944,386.629340,385.975089,385.685473,385.685473,385.685473,385.685473,385.685473,385.685473,385.685473,True
44739,200.0,197.556692,187.550387,207.118607,309.235253,175.757996,226.871078,179.364872,216.441157,182.632738,208.910538,False
19670A,200.0,192.675813,236.576050,177.022676,250.660058,NaN,NaN,NaN,NaN,NaN,NaN,False
19670B,200.0,332.199222,273.277797,352.301441,271.809700,334.758079,281.901306,305.336349,305.336349,305.336349,305.336349,True
12833,200.0,222.692132,226.933512,225.744806,225.235004,225.235004,225.235004,225.235004,225.235004,225.235004,225.235004,True
44858,200.0,357.487607,332.201387,317.607236,326.634769,326.634769,326.634769,326.634769,326.634769,326.634769,326.634769,True
PHL1811,200.0,286.585666,266.320496,268.232942,269.027191,269.027191,269.027191,269.027191,269.027191,269.027191,269.027191,True
